In [ ]:
from pathlib import Path
import re
import pickle
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.ndimage import distance_transform_edt, gaussian_filter1d, binary_fill_holes
from skimage.measure import label, regionprops
from skimage.graph import route_through_array
from skimage.morphology import binary_dilation, binary_erosion, skeletonize

from topostats.plottingfuncs import Colormap

colormap = Colormap()
CMAP = colormap.get_cmap()

VMIN = 0
VMAX = 4

In [ ]:
# SAMPLE_TYPE = "ON_SC"
# DATE_TO_READ_FROM = "2024-03-22"
# TODAY_DATE = datetime.now().strftime("%Y-%m-%d")
# DATA_DIR = Path(f"/users/sylvi/topo_data/hariborings/extracted_grains/cas9_{SAMPLE_TYPE}/{DATE_TO_READ_FROM}")
# SAVE_DIR = Path(f"/Users/sylvi/topo_data/hariborings/processed_grains/cas9_{SAMPLE_TYPE}/{TODAY_DATE}")
# SAVE_DIR.mkdir(exist_ok=True, parents=True)
# MAX_PX_TO_NM = 10.0
# PLOT_RESULTS = False
# MAX_GRAIN_NUMBER = 500

# FILE_PATH = DATA_DIR / "grain_dict.pkl"
# print(FILE_PATH)
# with open(FILE_PATH, "rb") as f:
#     grain_dicts = pickle.load(f)

# print(f"number of images for sample type [{SAMPLE_TYPE}] : {len(grain_dicts.keys())}")

# # Cut off the number of grains
# grain_dicts_sample = {}
# for i, (key, grain_dictionary) in enumerate(grain_dicts.items()):
#     if i > MAX_GRAIN_NUMBER:
#         break
#     grain_dicts_sample[key] = grain_dictionary

# print(f"number of images in sample for {SAMPLE_TYPE} : {len(grain_dicts_sample.keys())}")

# grain_dict = grain_dicts_sample

In [ ]:
TODAY_DATE = datetime.now().strftime("%Y-%m-%d")
DATE_TO_READ_FROM = "2024-03-22"
BASE_DATA_DIR = Path(f"/Users/sylvi/topo_data/hariborings/extracted_grains/")
data_dirs = {
    "ON_SC": BASE_DATA_DIR / f"cas9_ON_SC/{DATE_TO_READ_FROM}",
    "OT1_SC": BASE_DATA_DIR / f"cas9_OT1_SC/{DATE_TO_READ_FROM}",
    "OT2_SC": BASE_DATA_DIR / f"cas9_OT2_SC/{DATE_TO_READ_FROM}",
}
BASE_SAVE_DIR = Path(f"/Users/sylvi/topo_data/hariborings/processed_grains/")
MAX_PX_TO_NM = 10.0

save_dirs = {
    "ON_SC": BASE_SAVE_DIR / f"cas9_ON_SC/{TODAY_DATE}",
    "OT1_SC": BASE_SAVE_DIR / f"cas9_OT1_SC/{TODAY_DATE}",
    "OT2_SC": BASE_SAVE_DIR / f"cas9_OT2_SC/{TODAY_DATE}",
}
for sample_type in save_dirs.keys():
    save_dirs[sample_type].mkdir(exist_ok=True, parents=True)

sample_types = ["ON_SC", "OT1_SC", "OT2_SC"]
grains_dicts = {}
for sample_type in sample_types:
    grains_dicts[sample_type] = {}
    file_path = data_dirs[sample_type] / "grain_dict.pkl"
    with open(file_path, "rb") as f:
        sample_grains_dicts = pickle.load(f)
        for grain_index, single_grain_dict in sample_grains_dicts.items():
            p_to_nm = single_grain_dict["p_to_nm"]
            if p_to_nm > MAX_PX_TO_NM:
                print(
                    f"Skipping grain {grain_index} for sample type [{sample_type}] due to p_to_nm too large: {p_to_nm} > {MAX_PX_TO_NM}"
                )
            else:
                grains_dicts[sample_type][grain_index] = single_grain_dict


# print num grains for each sample type
for sample_type, grain_dict in grains_dicts.items():
    print(f"Number of grains for sample type [{sample_type}]: {len(grain_dict.keys())}")